In [1]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from environments import RodentSingleClipTrack

params = {
    "scale_factor": .9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 3,
    "clip_path": "12_21_1_250.h5",
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
    ],
}

In [2]:
env = RodentSingleClipTrack(params)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-M

In [3]:
type(env._ref_traj.quaternion)

numpy.ndarray

In [43]:
env.body_idxs

Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17], dtype=int32)

In [44]:
env._ref_traj.body_positions

array([[-0.05157034, -0.05444217, -0.05978363, ...,  0.04417997,
         0.0439658 ,  0.04394051],
       [ 0.34050909,  0.3449859 ,  0.35593086, ...,  0.12506871,
         0.12573494,  0.1262829 ],
       [ 0.07103304,  0.07158838,  0.07326452, ...,  0.06885403,
         0.0690574 ,  0.06955328],
       ...,
       [-0.06718285, -0.05690969, -0.0398935 , ...,  0.01185974,
         0.00975415, -0.00516812],
       [ 0.3718082 ,  0.38318591,  0.39113626, ...,  0.11357037,
         0.11351197,  0.11167984],
       [ 0.01327727,  0.01352061,  0.01167   , ...,  0.00936597,
         0.0110836 ,  0.01711685]])

In [45]:
rng = random.PRNGKey(0)
state = env.reset(rng)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx

AttributeError: 'jaxlib.xla_extension.XLACompatibleSharding' object has no attribute 'body_positions'

In [36]:
state.info['start_frame']

Array(56, dtype=int32)

In [39]:
env._ref_traj.velocity[:, state.info['start_frame']]

array([ 0.40821564, -0.20567179, -0.11344627])

In [40]:
state.obs

Array([ 4.51841280e-02,  5.42650037e-02,  6.40991330e-02,  7.40401298e-02,
        8.82725418e-02,  4.09621656e-01,  4.06745553e-01,  4.04145032e-01,
        4.02582884e-01,  4.00313556e-01,  7.11002350e-02,  6.97781369e-02,
        6.98768869e-02,  7.05022812e-02,  6.89362735e-02,  3.70198153e-02,
        4.13735092e-01,  7.33691603e-02,  9.71111119e-01, -2.00454928e-02,
       -5.62170558e-02, -2.31043264e-01, -1.32956237e-01,  2.32894346e-01,
        5.23598790e-01, -1.12722740e-01,  1.83160737e-01, -5.23596764e-01,
       -1.17349572e-01, -1.41577616e-01, -1.88900843e-01, -3.97443384e-01,
        1.20476425e-01,  0.00000000e+00,  9.42960009e-02, -1.06999695e-01,
        2.74080545e-01, -3.46574098e-01, -1.39625356e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  